In [ ]:
# Copyright (c) 2024 Houssem Ben Braiek, Emilio Rivera-Landos, IVADO, SEMLA
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import os
import os
from dotenv import load_dotenv
load_dotenv()
MLFLOW_URI = os.getenv("MLFLOW_URI")
import warnings
warnings.filterwarnings("ignore")
import mlflow
from sklearn.pipeline import Pipeline
from deepchecks.tabular import Dataset
from deepchecks.tabular.suites import full_suite
from deepchecks.tabular.suites import model_evaluation

In [ ]:
from flight_delay import pre_processing,train_utils,features
from flight_delay.mlflow.registry import load_model

In [ ]:
# dataset, dvc_info = load_dataset_from_localfs(splits_data_dir="../data/splits/reference")

In [ ]:
from flight_delay import pre_processing,train_utils,features
data_path = '../data/flights_delay_dataset.csv'
df_flights = pre_processing.load_data(data_path)
df_ref = df_flights[df_flights["flight_year"]==2019]
# df_ref.to_csv("../data/reference_data.csv")

In [ ]:
train_start = "2019-01-01"
train_end   = "2019-10-01"

test_start = "2019-10-01"
test_end   = "2020-01-01"

target_col = "ARR_DELAY"
split_col  = "FL_DATE"

X_train,y_train,X_test,y_test = train_utils.train_test_split_func(df_ref,train_start,train_end,test_start,test_end,split_col,target_col)

In [ ]:
model_name, model_version = "lightGBM", "1"
model_under_test = load_model(MLFLOW_URI, model_name, model_version)

In [ ]:
numerical_cols = features.numerical_cols
categorical_cols = features.categorical_cols
time_cols = features.time_cols
passthrough_cols = features.passthrough_cols

features_list = numerical_cols + categorical_cols + time_cols + passthrough_cols

predicted_var = "predicted_arrival_delay"

ds_train = Dataset(X_train[features_list], label=y_train, cat_features=categorical_cols)
ds_test =  Dataset(X_test[features_list],  label=y_test, cat_features=categorical_cols)

In [ ]:
suite = full_suite()
suite_result = suite.run(train_dataset=ds_train, 
                         test_dataset=ds_test, 
                         model=model_under_test)

In [ ]:
# suite_result.show()
suite_result.show_in_iframe()

In [ ]:
evaluation_suite = model_evaluation()
suite_result = suite.run(train_dataset=ds_train, 
                         test_dataset=ds_test, 
                         model=model_under_test)

In [ ]:
suite_result.show_in_iframe()